## 深入浅出pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'].insert(0, 'SimHei')
plt.rcParams['axes.unicode_minus'] = False
get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'svg'")

### 科比投篮数据分析

1. 科比使用得最多的投篮动作
2. 科比交手次数最多的球队
3. 科比有出手的比赛有多少场
4. 科比职业生涯（常规赛+季后赛）总得分（不含罚篮）
5. 科比得分最高的五场比赛（对手、投篮次数、得分、命中率）
6. 科比得分最多的三个赛季（赛季、投篮次数、得分、命中率）

In [ ]:
# 不限制最大显示的列数
pd.set_option('display.max_columns', None)

In [ ]:
# 加载科比投篮数据
kobe_df = pd.read_csv('res/科比投篮数据.csv', index_col='shot_id')
kobe_df.tail(5)

In [ ]:
kobe_df.info()

In [ ]:
# 科比使用得最多的投篮动作是什么
kobe_df.action_type.value_counts().index[0]

In [ ]:
kobe_df.groupby('action_type')['action_type'].count().nlargest(1).index[0]

In [ ]:
# 科比交手次数最多的球队是哪支队伍
kobe_df.drop_duplicates('game_id').opponent.value_counts().index[0]

In [ ]:
kobe_df.drop_duplicates('game_id').groupby('opponent').opponent.count().idxmax()

In [ ]:
# 科比有出手的比赛有多少场
kobe_df.game_id.nunique()

In [ ]:
# 统计科比常规赛和季后赛的投篮命中率
temp = kobe_df.dropna().pivot_table(index=['playoffs', 'shot_type'], columns=['shot_made_flag'], values='game_id', aggfunc='count')
temp = temp.divide(temp.sum(axis=1), axis=0)
temp

In [ ]:
# 填充shot_made_flag字段的缺失值
def handle(x):
    playoffs, shot_type, shot_made_flag = x
    if np.isnan(shot_made_flag):
        shot_made_flag = 1 if np.random.random() < temp.at[(playoffs, shot_type), 1.0] else 0
    return shot_made_flag


kobe_df['shot_made_flag'] = kobe_df[['playoffs', 'shot_type', 'shot_made_flag']].apply(handle, axis=1).astype('?')
kobe_df.info()

In [ ]:
# 处理得分字段
kobe_df['point'] = kobe_df.shot_type.str[0].astype('i8')
kobe_df['point'] = kobe_df[['shot_made_flag', 'point']].apply(lambda x: x.loc['point'] if x.loc['shot_made_flag'] else 0, axis=1)
kobe_df

In [ ]:
# 参考数据：投篮命中数11719
kobe_df.shot_made_flag.sum()

In [ ]:
# 参考数据：不含罚篮的投篮得分25265
kobe_df.point.sum()

In [ ]:
# 科比得分最多的赛季是哪个赛季和分数
kobe_df.groupby('season').point.sum().nlargest(3)

In [ ]:
# 获得得分最高的5场比赛的game_id
index = kobe_df.groupby('game_id').point.sum().nlargest(5).index.values
index

In [ ]:
# 用布尔索引筛选数据
kobe_df[np.in1d(kobe_df.game_id, index)]

In [ ]:
# 用query方法筛选数据
kobe_df.query('game_id in @index')

In [ ]:
# 科比得分最高的五场比赛（对手、投篮次数、得分、命中率）
# 参考数据（含罚篮）：TOR - 81分 / POR - 65分 / DAL - 62分 / NYK - 61分 / MEM - 60分 / UTA - 60分
df1 = kobe_df[np.in1d(kobe_df.game_id, index)].groupby(
    'game_id'
)[['game_date', 'opponent', 'game_id', 'shot_made_flag', 'point']].agg({
    'game_date': 'max',
    'opponent': 'max',
    'game_id': 'count',
    'shot_made_flag': 'sum',
    'point': 'sum'
})
df1['rate'] = df1.shot_made_flag / df1.game_id
df1.drop(columns=['shot_made_flag'], inplace=True)
df1.reset_index(drop=True, inplace=True)
df1.set_index('game_date', inplace=True)
df1.rename(columns={'opponent': '对手', 'game_id': '出手次数', 'point': '得分', 'rate': '命中率'}, inplace=True)
df1.sort_values(by='得分', ascending=False).style.format(formatter={'命中率': '{:.2%}'})

In [ ]:
df2 = kobe_df.query('game_id in @index').groupby(
    'game_id'
)[['game_date', 'opponent', 'game_id', 'shot_made_flag', 'point']].agg({
    'game_date': 'max',
    'opponent': 'max',
    'game_id': 'count',
    'shot_made_flag': 'sum',
    'point': 'sum'
})
df2['rate'] = df2.shot_made_flag / df2.game_id
df2.drop(columns=['shot_made_flag'], inplace=True)
df2.reset_index(drop=True, inplace=True)
df2.set_index('game_date', inplace=True)
df2.rename(columns={'opponent': '对手', 'game_id': '出手次数', 'point': '得分', 'rate': '命中率'}, inplace=True)
df2.sort_values(by='得分', ascending=False).style.format(formatter={'命中率': '{:.2%}'})

In [ ]:
# 科比得分最多的三个赛季（赛季、投篮次数、得分、命中率）


### 深圳二手房数据分析

1. 统计深圳二手房单价分布规律
2. 统计深圳二手房总价分布规律
3. 统计每个区总价和均价的均值
4. 深圳每个区单价Top3的商圈
5. 哪种户型的二手房数量最多
6. 总价Top10的二手房分布在哪些区

In [ ]:
sz_df = pd.read_csv('res/深圳二手房数据.csv')
sz_df.info()

In [ ]:
sz_df.drop(columns='Unnamed: 0', inplace=True)
sz_df.info()

In [ ]:
# 修正列名
sz_df.rename(columns={'hourseType': 'house_type', 'hourseSize': 'house_size'}, inplace=True)
sz_df.info()

In [ ]:
# 将tax字段处理为bool类型
sz_df['tax'] = sz_df.tax.fillna('').astype('?')
sz_df.info()

In [ ]:
sz_df.head(5)

In [ ]:
# 获取描述性统计信息
sz_df.total_price.agg(['mean', 'max', 'min', 'skew', 'kurt'])

In [ ]:
sz_df.unit_price.agg(['mean', 'max', 'min', 'skew', 'kurt'])

In [ ]:
sz_df.house_size.agg(['mean', 'max', 'min', 'skew', 'kurt'])

In [ ]:
sz_df[sz_df.unit_price < 10000]

In [ ]:
# 删除异常数据（单价小于10000）
sz_df.drop(index=sz_df[sz_df.unit_price < 10000].index, inplace=True)
sz_df.shape

In [ ]:
# 删除面积在10平米以下200平米以上的房屋信息
sz_df.drop(index=sz_df.query('house_size < 10 or house_size > 200').index, inplace=True)
sz_df.shape

In [ ]:
# 添加一个总房间数字段
sz_df['rooms_num'] = sz_df.house_type.str.extract('(\d+)室(\d+)厅').astype('i8').sum(axis=1)
sz_df.rooms_num.describe()

In [ ]:
sz_df.tail(5)

In [ ]:
# 删除房间总数大于8个的房屋信息
sz_df.drop(index=sz_df[sz_df.rooms_num > 8].index, inplace=True)
sz_df.shape

In [ ]:
# 单价分布
sz_df.unit_price.plot(kind='hist', figsize=(9, 5), bins=15, ylabel='')
plt.xticks(np.arange(10000, 210001, 20000))
plt.show()

In [ ]:
# 总价分布
sz_df.total_price.plot(kind='hist', figsize=(9, 5), bins=15, ylabel='')
plt.xticks(np.arange(100, 2901, 400))
plt.show()

In [ ]:
# 统计每个区总价和均价的均值
sz_df.pivot_table(
    index='area',
    values=['title', 'unit_price', 'total_price'],
    aggfunc={'title': 'count', 'unit_price': 'mean', 'total_price': 'mean'}
).round(1).sort_values(
    'unit_price', ascending=False
).style.format(
    formatter={
        'total_price': '￥{:.0f}万元',
        'unit_price': '￥{:,.0f}元'
    }
)

In [ ]:
# 深圳每个区房屋平均单价Top3商圈
temp_df = sz_df.groupby(['area', 'position'])[['unit_price']].mean().round(1)
temp_df['rank'] = temp_df.unit_price.groupby('area').rank(method='dense', ascending=False).astype('i8')
temp_df.query('rank <= 3')

In [ ]:
# 深圳每个区房屋平均单价Top3商圈
temp_df = sz_df.groupby(['area', 'position'], as_index=False)[['unit_price']].mean().round(1)
temp_df = temp_df.groupby('area')[['position', 'unit_price']].apply(lambda x: x.nlargest(3, 'unit_price'))
temp_df.style.hide(level=1).format(formatter={'unit_price': '￥{:,.0f}元'})

In [ ]:
# 哪种户型的二手房数量最多
sz_df.groupby('house_type').house_type.count().nlargest(1).index[0]

In [ ]:
# 总价Top10的二手房分布在哪些区
top10 = sz_df.total_price.nlargest(10).index.values
# 通过花式索引获取对应的行
sz_df.loc[top10].groupby('area').area.count()

### 销售利润下滑诊断分析

In [ ]:
detail_df = pd.read_excel('res/商品销售明细表.xlsx', sheet_name='Sheet1')
outlet_df = pd.read_excel('res/门店信息维度表.xlsx', sheet_name='Sheet1')
commod_df = pd.read_excel('res/商品信息维度表.xlsx', sheet_name='Sheet1')

In [ ]:
detail_df.info()

In [ ]:
detail_df.rename(columns={'日期(年月日)': '销售日期'}, inplace=True)
detail_df['销售日期'] = pd.to_datetime(detail_df.销售日期)
detail_df['月份'] = detail_df.销售日期.dt.month
detail_df['利润额'] = detail_df.销售额 - detail_df.成本额
detail_df.head(5)

In [ ]:
temp1 = detail_df.groupby('月份')[['销售额', '利润额']].sum()
temp1['销售月环比'] = temp1.销售额.pct_change()
temp1['利润月环比'] = temp1.利润额.pct_change()
temp1[['销售额', '销售月环比', '利润额', '利润月环比']].style.format(
    formatter={
        '销售月环比': '{:.2%}',
        '利润月环比': '{:.2%}',
    },
    na_rep='--------'
)

In [ ]:
temp1.plot(kind='line', figsize=(9, 5), xlabel='', y=['销售额', '利润额'], color=['navy', 'coral'], marker='o')
plt.ylim(0, 1.4e7)
plt.grid(axis='y', linestyle=':', alpha=0.5)
plt.show()

In [ ]:
import matplotlib.ticker as tkr

ax = temp1.销售额.plot(kind='line', figsize=(9, 5), marker='o', color='navy', linestyle='--')
temp1.利润额.plot(ax=ax, kind='line', marker='*', color='darkgreen', linestyle='--', xlabel='')
plt.ylim(0, 14000000)
plt.legend(loc='lower right')

# 基于ax构建双胞胎坐标系（共享横轴，自己定制纵轴）
ax2 = ax.twinx()
ax2.yaxis.set_major_formatter(tkr.PercentFormatter(xmax=1, decimals=0))
profs_rates = temp1.利润额 / temp1.销售额
profs_rates.plot(ax=ax2, kind='line', marker='^', color='r', linestyle=':', label='毛利率')
plt.ylim(0.45, 0.65)
plt.legend()
plt.grid(axis='y', linestyle=':', alpha=0.5)
plt.show()

In [ ]:
# 事实表连接维度表
merged_df = pd.merge(detail_df, outlet_df, how='left', on='门店编码')
merged_df = pd.merge(merged_df, commod_df, how='left', on='商品编码')
august_df = merged_df.query('月份 == 8')
august_df

In [ ]:
temp_df2 = august_df.groupby('省份')[['销售额', '成本额']].sum()
temp_df2.nlargest(10, '成本额')

In [ ]:
temp_df2.nlargest(10, '成本额').plot(kind='bar', figsize=(9, 5), xlabel='')
plt.xticks(rotation=0)
plt.show()

In [ ]:
temp_df3 = august_df.query('省份 == "湖南省"').groupby('城市')[['销售额', '成本额']].sum()
temp_df3['利润率'] = (temp_df3.销售额 - temp_df3.成本额) / temp_df3.销售额
temp_df3.nsmallest(3, '利润率').style.format(formatter={'利润率': '{:.2%}'})

In [ ]:
temp_df4 = august_df.query('省份 == "湖南省" and 城市 == "长沙市"').groupby('门店名称')[['销售额', '成本额']].sum()
temp_df4['利润率'] = (temp_df4.销售额 - temp_df4.成本额) / temp_df4.销售额
temp_df4.sort_values(by='利润率').style.format(formatter={'利润率': '{:.2%}'})

In [ ]:
august_df = august_df.query('省份 == "湖南省" and 城市 == "长沙市" and 门店名称 == "长沙梅溪湖店"')
august_df.shape

In [ ]:
temp_df5 = august_df.groupby('商品类别')[['销售额', '成本额']].sum()
temp_df5['利润率'] = (temp_df5.销售额 - temp_df5.成本额) / temp_df5.销售额
temp_df5.sort_values(by='利润率').style.format(formatter={'利润率': '{:.2%}'})

In [ ]:
temp_df6 = august_df.query('商品类别 == "零食"').groupby('商品名称')[['销售额', '成本额']].sum()
temp_df6['利润率'] = (temp_df6.销售额 - temp_df6.成本额) / temp_df6.销售额
temp_df6.sort_values(by='利润率').style.format(formatter={'利润率': '{:.2%}'})